In [12]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

In [55]:
# read the Wikipedia table
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=data[0]

# set column names
df.columns=['PostalCode','Borough','Neighbourhood']


In [57]:
# Remove rows in Borough where value is Not assigned
df = df[df.Borough != 'Not assigned']


In [59]:
# Group any postcode duplicates in rows together
df=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))


In [64]:
# shape of the dataframe
df.shape

(103, 2)